In [1]:
import numpy as np
import pandas as pd

from IPython.display import clear_output

In [2]:
train = pd.read_csv('../data/train.csv', usecols=['idzona'])
test = pd.read_csv('../data/test.csv', usecols=['idzona'])

train_places = pd.read_csv('../data/features/ciudad_prov_lat_lng_V2_train.csv', usecols=['provincia', 'ciudad'])
test_places = pd.read_csv('../data/features/ciudad_prov_lat_lng_V2_test.csv', usecols=['provincia', 'ciudad'])

In [7]:
train = train.join(train_places)
test = test.join(test_places)

# Primero se rellena el train

In [10]:
modas_ciudades_train = train.dropna().groupby('ciudad').agg(lambda x:x.value_counts().index[0])
modas_provincias_train = train.dropna().groupby('provincia').agg(lambda x:x.value_counts().index[0])

modas_ciudades_train.drop(columns=['provincia'], inplace=True)
modas_provincias_train.drop(columns=['ciudad'], inplace=True)

In [11]:
modas_provincias_train.head(5)

,idzona
provincia,
Aguascalientes,282.0
Baja California Norte,2351.0
Baja California Sur,253921.0
Campeche,4559.0
Chiapas,9047.0


In [12]:
modas_ciudades_train.head(5)

,idzona
ciudad,
Abasolo,33537.0
Acambay,282627.0
Acapulco de Juárez,38062.0
Acatic,48686.0
Acatzingo,82619.0


In [15]:
indice1 = train.loc[train['idzona'].isnull()].index
indice1.size

28621

In [16]:
%%time
ini = 0
fin = indice1.size
i=1
for index in indice1[ini:fin] :
    clear_output(wait=True)
    # La condición está porque hay ciudades que no registran idzona.
    if(train.loc[index, 'ciudad'] in modas_ciudades_train.index):
        train.loc[index, 'idzona'] = modas_ciudades_train.loc[train.loc[index, 'ciudad'], 'idzona']
    print(i, 'de', fin - ini)
    i = i + 1

28621 de 28621
CPU times: user 2min 10s, sys: 11.7 s, total: 2min 21s
Wall time: 1min 58s


In [17]:
# Se revisa cuántos quedan nulos
train.isnull().sum()
# Estos se rellenan con las modas de las provincias

idzona       148
ciudad         0
provincia      0
dtype: int64

In [18]:
# Primero se ubican estos casos (sus índices)
indice2 = train.loc[train['idzona'].isnull()].index
indice2.size

148

In [19]:
%%time
ini = 0
fin = indice2.size
i=1
for index in indice2[ini:fin] :
    clear_output(wait=True)
    train.loc[index, 'idzona'] = modas_provincias_train.loc[train.loc[index, 'provincia'], 'idzona']
    print(i, 'de', fin - ini)
    i = i + 1

148 de 148
CPU times: user 542 ms, sys: 83.1 ms, total: 625 ms
Wall time: 526 ms


In [21]:
train.isnull().sum()

idzona       0
ciudad       0
provincia    0
dtype: int64

# Ahora se rellena el test

In [22]:
modas_ciudades_test = test.dropna().groupby('ciudad').agg(lambda x:x.value_counts().index[0])
modas_provincias_test = test.dropna().groupby('provincia').agg(lambda x:x.value_counts().index[0])

modas_ciudades_test.drop(columns=['provincia'], inplace=True)
modas_provincias_test.drop(columns=['ciudad'], inplace=True)

In [23]:
modas_provincias_test.head(5)

,idzona
provincia,
Aguascalientes,882.0
Baja California Norte,2351.0
Baja California Sur,253921.0
Campeche,4559.0
Chiapas,9292.0


In [24]:
modas_ciudades_test.head(5)

,idzona
ciudad,
Abasolo,33537.0
Acambay,52278.0
Acaponeta,67486.0
Acapulco de Juárez,38062.0
Acatic,48691.0


In [26]:
indice3 = test.loc[test['idzona'].isnull()].index
indice3.size

7179

In [27]:
%%time
ini = 0
fin = indice3.size
i=1
for index in indice3[ini:fin] :
    clear_output(wait=True)
    # La condición está porque hay ciudades que no registran idzona.
    if(test.loc[index, 'ciudad'] in modas_ciudades_test.index):
        test.loc[index, 'idzona'] = modas_ciudades_test.loc[test.loc[index, 'ciudad'], 'idzona']
    print(i, 'de', fin - ini)
    i = i + 1

7179 de 7179
CPU times: user 20.1 s, sys: 2.66 s, total: 22.8 s
Wall time: 19 s


In [28]:
# Se revisa cuántos quedan nulos
test.isnull().sum()
# Estos se rellenan con las modas de las provincias

idzona       65
ciudad        0
provincia     0
dtype: int64

In [29]:
# Primero se ubican estos casos (sus índices)
indice4 = test.loc[test['idzona'].isnull()].index
indice4.size

65

In [30]:
%%time
ini = 0
fin = indice4.size
i=1
for index in indice4[ini:fin] :
    clear_output(wait=True)
    test.loc[index, 'idzona'] = modas_provincias_test.loc[test.loc[index, 'provincia'], 'idzona']
    print(i, 'de', fin - ini)
    i = i + 1

65 de 65
CPU times: user 195 ms, sys: 26.4 ms, total: 222 ms
Wall time: 185 ms


In [31]:
test.isnull().sum()

idzona       0
ciudad       0
provincia    0
dtype: int64

# Se pueden quitar las columnas 'ciudad' y 'provincia'

In [34]:
train.drop(columns=['ciudad', 'provincia'], inplace=True)
test.drop(columns=['ciudad', 'provincia'], inplace=True)

In [35]:
train.to_csv('../data/id_zona_V2_train.csv', index=False)

In [38]:
prueba = pd.read_csv('../data/id_zona_V2_train.csv')
prueba

,idzona
0,23533.0
1,24514.0
2,48551.0
3,53666.0
4,47835.0
...,...
239995,53666.0
239996,51954.0
239997,50003995.0
239998,24162.0


In [39]:
test.to_csv('../data/id_zona_V2_test.csv', index=False)

In [40]:
prueba = pd.read_csv('../data/id_zona_V2_test.csv')
prueba

,idzona
0,275358.0
1,113851.0
2,23620.0
3,129347.0
4,57125.0
...,...
59995,275741.0
59996,57474.0
59997,72224.0
59998,83960.0
